In [671]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from tqdm import tqdm
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from scipy import sparse
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import nltk 
from sklearn.ensemble import BaggingClassifier

%matplotlib inline

In [672]:
ls data

contest2/


In [673]:
train = pd.read_csv('./data/contest2/train.csv', index_col=0)
train[:3]

,author,content,sentiment
26898,richardepryor,"@treasaint salad stuff, some chillis, whatever...",happiness
27635,reese,"@sunnyjamiel sunny, I'm a workin' on it. It's ...",neutral
3036,mutedriposte,@jolynnchew so early??,surprise


In [674]:
test = pd.read_csv('./data/contest2/test.csv', index_col=0)
test[:3]

,author,content
id,,
32823,valicast,Good Morning
16298,btb103,I just put my computer up on craigslist. I've ...
28505,anavil,in ten minutes shopping demi lovato-back aro...


In [676]:
transformer = dict(zip(np.unique(train['sentiment']), range(len(np.unique(train['sentiment'])))))
transformer

{'anger': 0,
 'boredom': 1,
 'empty': 2,
 'enthusiasm': 3,
 'fun': 4,
 'happiness': 5,
 'hate': 6,
 'love': 7,
 'neutral': 8,
 'relief': 9,
 'sadness': 10,
 'surprise': 11,
 'worry': 12}

In [677]:
X = train.drop(['sentiment'], axis=1)
y = train.sentiment.apply(lambda x: transformer[x])
X_kaggle = test

In [678]:
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [680]:
stemmer = PorterStemmer()

### tfidf

In [681]:
X_upd = []
for row in tqdm(X.values):
    r = ' '.join(row)
    r_array = r.split()
    r_array = list(map(lambda x: stemmer.stem(x), r_array))
    X_upd.append(' '.join(r_array))
    
X = np.array(X_upd)

100%|██████████| 30000/30000 [00:06<00:00, 4775.90it/s]


In [682]:
X_upd_test = []
for row in tqdm(X_kaggle.values):
    r = ' '.join(row)
    r_array = r.split()
    r_array = list(map(lambda x: stemmer.stem(x), r_array))
    X_upd_test.append(' '.join(r_array))
X_kaggle_tfidf = np.array(X_upd_test)

100%|██████████| 10000/10000 [00:02<00:00, 4281.63it/s]


In [683]:
count = CountVectorizer(ngram_range=(1, 4))
tfidf = TfidfVectorizer(ngram_range=(1, 5))

In [508]:
tfidf_train_features = tfidf.fit_transform(X, y)
tfidf_kaggle_features = tfidf.transform(X_kaggle_tfidf)

In [684]:
count_train_features = count.fit_transform(X, y)
count_kaggle_features = count.transform(X_kaggle_tfidf)

In [630]:
count_train_features.shape, tfidf_train_features.shape

((30000, 563741), (30000, 1147406))

In [511]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, SGDClassifier

In [722]:
svm = LogisticRegression(C=0.7, random_state=42, max_iter=100, multi_class='multinomial', solver='lbfgs', n_jobs=-1)

In [777]:
y_kaggle_pred_logreg = svm.predict(count_kaggle_features)

In [768]:
ss = pd.read_csv('./data/contest2/sampleSubmission.csv')

In [769]:
inverse_transformer = {
    'anger': 0,
    'boredom': 1,
    'empty': 2,
    'enthusiasm': 3,
    'fun': 4,
    'happiness': 5,
    'hate': 6,
    'love': 7,
    'neutral': 8,
    'relief': 9,
    'sadness': 10,
    'surprise': 11,
    'worry': 12
}
inverse_transformer = dict(zip(range(13), inverse_transformer.keys()))

In [770]:
y_pred_kaggle = [inverse_transformer[x] for x in y_pred_kaggle]

In [771]:
ss['sentiment'] = y_pred_kaggle

In [772]:
ss.index = ss['id']

In [773]:
ss = ss.drop('id', axis=1)

In [774]:
ss.to_csv('ss_expected_36.csv')